In [1]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
from collections import deque
import heapq
import math


class PathfindingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Polish Cities Robot Delivery Pathfinding")
        self.root.geometry("1400x900")
        self.root.configure(bg="#f0f0f0")
        
        # Graph data from diagram (a) - straight-line distances
        self.graph_a = {
            'Glogów': [('Leszno', 40), ('Wrocław', 89)],
            'Leszno': [('Glogów', 40), ('Poznań', 67), ('Wrocław', 87), ('Kalisz', 103)],
            'Poznań': [('Leszno', 67), ('Bydgoszcz', 108), ('Konin', 107)],
            'Bydgoszcz': [('Poznań', 108), ('Włocławek', 102), ('Konin', 90)],
            'Włocławek': [('Bydgoszcz', 102), ('Płock', 44)],
            'Płock': [('Włocławek', 44), ('Warsaw', 95)],
            'Konin': [('Poznań', 107), ('Bydgoszcz', 90), ('Łódź', 95), ('Kalisz', 80)],
            'Kalisz': [('Leszno', 103), ('Konin', 80), ('Wrocław', 128), ('Częstochowa', 128)],
            'Wrocław': [('Glogów', 89), ('Leszno', 87), ('Kalisz', 128), ('Opole', 80)],
            'Łódź': [('Konin', 95), ('Warsaw', 118), ('Radom', 124), ('Częstochowa', 107)],
            'Warsaw': [('Płock', 95), ('Łódź', 118), ('Radom', 91)],
            'Radom': [('Warsaw', 91), ('Łódź', 124), ('Kielce', 70), ('Częstochowa', 190)],
            'Częstochowa': [('Kalisz', 128), ('Łódź', 107), ('Radom', 190), ('Katowice', 61), ('Opole', 90)],
            'Opole': [('Wrocław', 80), ('Częstochowa', 90), ('Katowice', 68)],
            'Katowice': [('Opole', 68), ('Częstochowa', 61), ('Kraków', 68)],
            'Kielce': [('Radom', 70), ('Kraków', 102)],
            'Kraków': [('Katowice', 68), ('Kielce', 102)]
        }
        self.heuristic_b = {
            'Glogów': 45,
            'Leszno': 140,
            'Poznań': 90,
            'Bydgoszcz': 110,
            'Włocławek': 53,
            'Płock': 0,
            'Konin': 120,
            'Kalisz': 120,
            'Wrocław': 100,
            'Łódź': 165,
            'Warsaw': 130,
            'Radom': 82,
            'Częstochowa': 280,
            'Opole': 118,
            'Katowice': 80,
            'Kielce': 120,
            'Kraków': 85
        }
        
        self.start_city = 'Glogów'
        self.goal_city = 'Płock'
        
        self.setup_ui()
    
    def setup_ui(self):
        """Setup the user interface"""
        
        # Title
        title_frame = tk.Frame(self.root, bg="#34495e", pady=15)
        title_frame.pack(fill=tk.X)
        
        tk.Label(
            title_frame,
            text="Polish Cities Robot Delivery Pathfinding",
            font=("Arial", 20, "bold"),
            bg="#34495e",
            fg="white"
        ).pack()
        
        tk.Label(
            title_frame,
            text="Start: Glogów (Blue) → Goal: Płock (Red)",
            font=("Arial", 12),
            bg="#34495e",
            fg="#ecf0f1"
        ).pack()
        
        # Main container
        main_container = tk.PanedWindow(self.root, orient=tk.HORIZONTAL, bg="#f0f0f0")
        main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left panel - Controls and Results
        left_panel = tk.Frame(main_container, bg="#f0f0f0")
        main_container.add(left_panel, width=700)
        
        # Control Panel
        control_frame = tk.LabelFrame(
            left_panel,
            text="Algorithm Selection",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        control_frame.pack(fill=tk.X, padx=5, pady=5)
        
        tk.Button(
            control_frame,
            text="1. Depth-First Search (DFS)",
            command=lambda: self.run_algorithm('DFS'),
            bg="#3498db",
            fg="white",
            font=("Arial", 11, "bold"),
            pady=8,
            cursor="hand2"
        ).pack(fill=tk.X, pady=3)
        
        tk.Button(
            control_frame,
            text="2. Breadth-First Search (BFS)",
            command=lambda: self.run_algorithm('BFS'),
            bg="#27ae60",
            fg="white",
            font=("Arial", 11, "bold"),
            pady=8,
            cursor="hand2"
        ).pack(fill=tk.X, pady=3)
        
        tk.Button(
            control_frame,
            text="3. A* Search Algorithm",
            command=lambda: self.run_algorithm('A*'),
            bg="#e74c3c",
            fg="white",
            font=("Arial", 11, "bold"),
            pady=8,
            cursor="hand2"
        ).pack(fill=tk.X, pady=3)
        
        tk.Button(
            control_frame,
            text="Clear All",
            command=self.clear_all,
            bg="#95a5a6",
            fg="white",
            font=("Arial", 11),
            pady=8,
            cursor="hand2"
        ).pack(fill=tk.X, pady=3)
        
        # Results Panel
        results_frame = tk.LabelFrame(
            left_panel,
            text="Algorithm Results",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        results_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Create text widget for results
        self.results_text = scrolledtext.ScrolledText(
            results_frame,
            font=("Courier", 10),
            wrap=tk.WORD,
            bg="#fafafa",
            height=15
        )
        self.results_text.pack(fill=tk.BOTH, expand=True)
        
        # State Space Panel
        state_frame = tk.LabelFrame(
            left_panel,
            text="State Space (Open & Closed Containers)",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        state_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.state_text = scrolledtext.ScrolledText(
            state_frame,
            font=("Courier", 9),
            wrap=tk.WORD,
            bg="#fafafa",
            height=12
        )
        self.state_text.pack(fill=tk.BOTH, expand=True)
        
        # Right panel - Comparison
        right_panel = tk.Frame(main_container, bg="#f0f0f0")
        main_container.add(right_panel)
        
        # Comparison Panel
        comparison_frame = tk.LabelFrame(
            right_panel,
            text="Algorithm Comparison & Analysis",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        comparison_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.comparison_text = scrolledtext.ScrolledText(
            comparison_frame,
            font=("Courier", 9),
            wrap=tk.WORD,
            bg="#fafafa"
        )
        self.comparison_text.pack(fill=tk.BOTH, expand=True)
        
        # Add initial comparison text
        self.show_initial_comparison()
    
    def show_initial_comparison(self):
        """Show initial comparison of algorithms"""
        text = """
╔════════════════════════════════════════════════════════════════════╗
║          ALGORITHM COMPARISON & ADVANTAGES/DISADVANTAGES           ║
╚════════════════════════════════════════════════════════════════════╝

1. DEPTH-FIRST SEARCH (DFS)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✓ ADVANTAGES:
  • Memory efficient - uses less memory (only stores path)

  • Requires good heuristic function
  • More complex to implement
  • Memory usage depends on heuristic quality
  • Heuristic calculation adds overhead

Formula: f(n) = g(n) + h(n)
  • g(n) = actual cost from start to node n
  • h(n) = estimated cost from n to goal (heuristic)
Data Structure: Priority Queue
Completeness: Complete
Optimality: Optimal (with admissible heuristic)


CONTEXT: Polish Cities Robot Delivery Problem
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start: Glogów (Blue node)
Goal: Płock (Red node)

• DFS may find A path, but not necessarily the shortest
• BFS will find shortest path by number of cities visited
• A* will find the optimal path considering actual distances
  using the straight-line heuristic from diagram (b)

Run each algorithm to see the differences!
        """
        self.comparison_text.insert(tk.END, text)
    
    def clear_all(self):
        """Clear all text areas"""
        self.results_text.delete(1.0, tk.END)
        self.state_text.delete(1.0, tk.END)
    
    def log_result(self, message):
        """Log to results area"""
        self.results_text.insert(tk.END, message + "\n")
        self.results_text.see(tk.END)
        self.root.update_idletasks()
    
    def log_state(self, message):
        """Log to state space area"""
        self.state_text.insert(tk.END, message + "\n")
        self.state_text.see(tk.END)
        self.root.update_idletasks()
    
    def dfs(self):
        """Depth-First Search Algorithm"""
        self.log_result("="*70)
        self.log_result("DEPTH-FIRST SEARCH (DFS) ALGORITHM")
        self.log_result("="*70)
        self.log_result(f"Start: {self.start_city} → Goal: {self.goal_city}\n")
        
        # Initialize
        stack = [(self.start_city, [self.start_city], 0)]  # (node, path, cost)
        visited = set()
        step = 0
        
        self.log_state("="*70)
        self.log_state("DFS STATE SPACE EXPLORATION")
        self.log_state("="*70)
        self.log_state("Stack (Open Container): nodes to explore")
        self.log_state("Visited (Closed Container): explored nodes\n")
        
        while stack:
            step += 1
            current, path, cost = stack.pop()
            
            self.log_state(f"\n--- Step {step} ---")
            self.log_state(f"Current Node: {current}")
            self.log_state(f"Current Path: {' → '.join(path)}")
            self.log_state(f"Current Cost: {cost}")
            
            if current in visited:
                self.log_state(f"Already visited {current}, skipping...")
                continue
            
            visited.add(current)
            self.log_state(f"Closed Container: {sorted(visited)}")
            
            # Check if goal reached
            if current == self.goal_city:
                self.log_result(f"✓ GOAL REACHED!\n")
                self.log_result(f"Path Found: {' → '.join(path)}")
                self.log_result(f"Total Distance: {cost} km")
                self.log_result(f"Cities Visited: {len(path)}")
                self.log_result(f"Nodes Explored: {len(visited)}")
                self.log_state(f"\n✓ Goal '{self.goal_city}' reached!")
                return path, cost, len(visited)
            
            # Add neighbors to stack (in reverse for DFS left-to-right)
            neighbors = self.graph_a.get(current, [])
            self.log_state(f"Neighbors of {current}: {[n[0] for n in neighbors]}")
            
            for neighbor, distance in reversed(neighbors):
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    new_cost = cost + distance
                    stack.append((neighbor, new_path, new_cost))
            
            self.log_state(f"Open Container (Stack): {[s[0] for s in stack]}")
        
        self.log_result("✗ No path found!")
        return None, 0, len(visited)
    
    def bfs(self):
        """Breadth-First Search Algorithm"""
        self.log_result("="*70)
        self.log_result("BREADTH-FIRST SEARCH (BFS) ALGORITHM")
        self.log_result("="*70)
        self.log_result(f"Start: {self.start_city} → Goal: {self.goal_city}\n")
        
        # Initialize
        queue = deque([(self.start_city, [self.start_city], 0)])  # (node, path, cost)
        visited = set()
        step = 0
        
        self.log_state("="*70)
        self.log_state("BFS STATE SPACE EXPLORATION")
        self.log_state("="*70)
        self.log_state("Queue (Open Container): nodes to explore (FIFO)")
        self.log_state("Visited (Closed Container): explored nodes\n")
        
        while queue:
            step += 1
            current, path, cost = queue.popleft()
            
            self.log_state(f"\n--- Step {step} ---")
            self.log_state(f"Current Node: {current}")
            self.log_state(f"Current Path: {' → '.join(path)}")
            self.log_state(f"Current Cost: {cost}")
            
            if current in visited:
                self.log_state(f"Already visited {current}, skipping...")
                continue
            
            visited.add(current)
            self.log_state(f"Closed Container: {sorted(visited)}")
            
            # Check if goal reached
            if current == self.goal_city:
                self.log_result(f"✓ GOAL REACHED!\n")
                self.log_result(f"Path Found: {' → '.join(path)}")
                self.log_result(f"Total Distance: {cost} km")
                self.log_result(f"Cities Visited: {len(path)}")
                self.log_result(f"Nodes Explored: {len(visited)}")
                self.log_state(f"\n✓ Goal '{self.goal_city}' reached!")
                return path, cost, len(visited)
            
            # Add neighbors to queue
            neighbors = self.graph_a.get(current, [])
            self.log_state(f"Neighbors of {current}: {[n[0] for n in neighbors]}")
            
            for neighbor, distance in neighbors:
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    new_cost = cost + distance
                    queue.append((neighbor, new_path, new_cost))
            
            self.log_state(f"Open Container (Queue): {[q[0] for q in queue]}")
        
        self.log_result("✗ No path found!")
        return None, 0, len(visited)
    
    def a_star(self):
        """A* Search Algorithm"""
        self.log_result("="*70)
        self.log_result("A* SEARCH ALGORITHM")
        self.log_result("="*70)
        self.log_result(f"Start: {self.start_city} → Goal: {self.goal_city}")
        self.log_result(f"Heuristic: Straight-line distance to {self.goal_city}\n")
        
        # Initialize
        # Priority queue: (f_score, g_score, node, path)
        pq = [(self.heuristic_b[self.start_city], 0, self.start_city, [self.start_city])]
        visited = set()
        step = 0
        
        self.log_state("="*70)
        self.log_state("A* STATE SPACE EXPLORATION")
        self.log_state("="*70)
        self.log_state("Priority Queue (Open Container): nodes ordered by f(n)")
        self.log_state("f(n) = g(n) + h(n)")
        self.log_state("g(n) = actual cost from start")
        self.log_state("h(n) = heuristic (straight-line distance to goal)")
        self.log_state("Visited (Closed Container): explored nodes\n")
        
        while pq:
            step += 1
            f_score, g_score, current, path = heapq.heappop(pq)
            h_score = self.heuristic_b[current]
            
            self.log_state(f"\n--- Step {step} ---")
            self.log_state(f"Current Node: {current}")
            self.log_state(f"g({current}) = {g_score} (actual cost from start)")
            self.log_state(f"h({current}) = {h_score} (heuristic to goal)")
            self.log_state(f"f({current}) = {f_score} (total estimated cost)")
            self.log_state(f"Current Path: {' → '.join(path)}")
            
            if current in visited:
                self.log_state(f"Already visited {current}, skipping...")
                continue
            
            visited.add(current)
            self.log_state(f"Closed Container: {sorted(visited)}")
            
            # Check if goal reached
            if current == self.goal_city:
                self.log_result(f"✓ GOAL REACHED!\n")
                self.log_result(f"Optimal Path: {' → '.join(path)}")
                self.log_result(f"Total Distance: {g_score} km")
                self.log_result(f"Cities Visited: {len(path)}")
                self.log_result(f"Nodes Explored: {len(visited)}")
                self.log_state(f"\n✓ Goal '{self.goal_city}' reached with optimal path!")
                return path, g_score, len(visited)
            
            # Add neighbors to priority queue
            neighbors = self.graph_a.get(current, [])
            self.log_state(f"Neighbors of {current}:")
            
            for neighbor, distance in neighbors:
                if neighbor not in visited:
                    new_g = g_score + distance
                    new_h = self.heuristic_b[neighbor]
                    new_f = new_g + new_h
                    new_path = path + [neighbor]
                    
                    self.log_state(f"  {neighbor}: g={new_g}, h={new_h}, f={new_f}")
                    heapq.heappush(pq, (new_f, new_g, neighbor, new_path))
            
            # Show current priority queue
            pq_display = [(node, f"f={f}") for f, g, node, p in sorted(pq)[:5]]
            self.log_state(f"Open Container (top 5): {pq_display}")
        
        self.log_result("✗ No path found!")
        return None, 0, len(visited)
    
    def run_algorithm(self, algorithm):
        """Run selected algorithm"""
        self.clear_all()
        
        if algorithm == 'DFS':
            path, cost, nodes = self.dfs()
        elif algorithm == 'BFS':
            path, cost, nodes = self.bfs()
        elif algorithm == 'A*':
            path, cost, nodes = self.a_star()
        
        # Show summary comparison after running
        if path:
            self.log_result("\n" + "="*70)
            self.log_result(f"{algorithm} Algorithm Summary:")
            self.log_result("="*70)
            self.log_result(f"Algorithm Type: {algorithm}")
            self.log_result(f"Path: {' → '.join(path)}")
            self.log_result(f"Total Distance: {cost} km")
            self.log_result(f"Number of Cities: {len(path)}")
            self.log_result(f"Nodes Explored: {nodes}")
            self.log_result("="*70)


def main():
    """Main function"""
    root = tk.Tk()
    app = PathfindingApp(root)
    root.mainloop()


if __name__ == "__main__":
    main()